# Process Alpaca English and Translated Thai

In [2]:
import json
import pandas as pd
from datasets import (
    Dataset,
    load_dataset
)
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    HfArgumentParser,
    AdamW,
    DataCollatorForLanguageModeling,
)
from deepspeed.runtime.lr_schedules import WarmupDecayLR
from typing import Optional, Union, List, Dict, Any
import evaluate
from dataclasses import dataclass, field
import torch.nn as nn
import numpy as np
import wandb
import multiprocessing
import copy
cpu_cores = multiprocessing.cpu_count()



2023-04-01 13:13:06.591524: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 13:13:07.663430: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2023-04-01 13:13:07.663550: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2023-04-01 13:13:07.663564: W tensorflow/compiler/tf2tensorrt/utils/py_utils

In [3]:
# #original
# with open('../data_large/alpaca_data.json','r') as f:
#     data = json.load(f)
# dset = Dataset.from_list(data)

In [4]:
dset = load_dataset('yahma/alpaca-cleaned')
dset

Using custom data configuration yahma--alpaca-cleaned-3d288cdc05baa98a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/charipol/.cache/huggingface/datasets/yahma___json/yahma--alpaca-cleaned-3d288cdc05baa98a/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 51711
    })
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained('facebook/xglm-7.5B')
tokenizer('hey')

{'input_ids': [2, 30502], 'attention_mask': [1, 1]}

In [7]:
def preprocess_sft(example):
    if example['input']!="":
        example['text'] = f"<human>: {example['instruction']} <context>: {example['input']} <bot>: {example['output']}"
        example['has_context'] = 1
    else:
        example['text'] = f"<human>: {example['instruction']} <bot>: {example['output']}"
        example['has_context'] = 0
    example['nb_tokens'] = len(tokenizer(example['text'])['input_ids'])
    return example

dset_sft = dset.map(preprocess_sft,
                remove_columns=['instruction','input','output'])

  0%|          | 0/51711 [00:00<?, ?ex/s]

In [16]:
#99.9% are less than 512 tokens
pd.Series(dset_sft['train']['nb_tokens']).quantile([i/1000 for i in range(1000)])

0.000     14.00
0.001     17.00
0.002     19.00
0.003     19.00
0.004     20.00
          ...  
0.995    357.45
0.996    378.00
0.997    405.87
0.998    439.58
0.999    516.16
Length: 1000, dtype: float64

In [17]:
#only 37 examples are over 512; we decide to remove it first
(pd.Series(dset_sft['train']['nb_tokens'])>512).sum()

55

In [18]:
#filter down
dset_sft['train'] = dset_sft['train'].filter(lambda example: example["nb_tokens"]<=512)\
            .remove_columns(['nb_tokens'])

dset_sft

  0%|          | 0/52 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'has_context'],
        num_rows: 51656
    })
})

In [23]:
dset_sft['train'][120]

{'text': '<human>: Generate a summary of the given article. <context>: Between 1989 and 2019, the size of the U.S. economy, measured in terms of the gross domestic product (GDP), more than doubled. The growth was faster than in the prior two decades, driven in part by the dramatic expansion of the information technology sector in the 1990s and early 2000s, gains in worker productivity and strong consumer demand, and government investments in research and development. <bot>: The U.S. economy more than doubled in size between 1989 and 2019, largely due to the rise of the information technology sector, improvements in worker productivity and consumer demand, and government investments. This growth was faster than in the prior two decades.',
 'has_context': 1}

In [24]:
# dset_sft = dset_sft['train'].train_test_split(test_size=1000, seed=125)
# dset_sft

In [25]:
dset_sft.push_to_hub('pythainlp/alpaca_cleaned_en_sft', private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
dset_sft = load_dataset('pythainlp/alpaca_cleaned_en_sft')

Using custom data configuration pythainlp--alpaca_cleaned_en_sft-3609a06dc02a1421


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/51656 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/charipol/.cache/huggingface/datasets/pythainlp___parquet/pythainlp--alpaca_cleaned_en_sft-3609a06dc02a1421/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Load the value-head model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained('facebook/xglm-564M')
model = AutoModelForCausalLM.from_pretrained('facebook/xglm-564M')

# Preprocess the dataset.
def mask_labels(l, context_cue, human_cue, bot_cue):
    result = []
    i = 0
    while i < len(l):
        if (l[i:i+len(human_cue)] == human_cue)|((l[i:i+len(context_cue)] == context_cue)):
            while l[i:i+len(bot_cue)] != bot_cue:
                result.append(-100)
                i += 1
        else:
            result.append(l[i])
            i += 1
    return result
        
def preprocess_function(example):
    tokenized_qa = tokenizer(example['text']+tokenizer.eos_token, 
                            truncation=True, 
                            padding="max_length",
                            max_length=512,
                            add_special_tokens=False
                            )
    labels = copy.deepcopy(tokenized_qa['input_ids'])
    labels = mask_labels(labels, 
              tokenizer('<context>:', add_special_tokens=False)['input_ids'],
              tokenizer('<human>:', add_special_tokens=False)['input_ids'],
              tokenizer('<bot>:', add_special_tokens=False)['input_ids']
             )
    labels = [-100 if i==tokenizer.pad_token_id else i for i in labels]
    return {
        "input_ids": tokenized_qa["input_ids"],
        "attention_mask": tokenized_qa["attention_mask"],
        "labels": labels,
    }


Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

In [ ]:
tokenized_ds = dset_sft.map(preprocess_function, 
                      batched=False, 
                      num_proc=5, 
                      )